In [1]:
from elasticsearch import Elasticsearch
import json

In [2]:
es = Elasticsearch([{'host':'localhost', 'port':9200}])
es

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

In [3]:
e1={
    "first_name":"akash",
    "last_name":"rawat",
    "age": 30,
    "about": "Love to play tekken",
    "interests": ['movies','music'],
}

print(e1)

{'first_name': 'akash', 'last_name': 'rawat', 'age': 30, 'about': 'Love to play tekken', 'interests': ['movies', 'music']}


# Inserting a document:

In [4]:
res = es.index(index='corpltd', id=1, body=e1)

All the necessary administration tasks were taken care of in the background, using default values.

In [5]:
e2={
    "first_name" :  "ankit",
    "last_name" :   "rawat",
    "age" :         28,
    "about" :       "I like to collect rock albums",
    "interests":  [ "music" ]
}
e3={
    "first_name" :  "rajeev",
    "last_name" :   "rawat",
    "age" :         18,
    "about":        "I like to build cabinets",
    "interests":  [ "forestry" ]
}
res=es.index(index='corpltd',id=2,body=e2)
res=es.index(index='corpltd',id=3,body=e3)
print(res)

{'_index': 'corpltd', '_type': '_doc', '_id': '3', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 13, '_primary_term': 1}


# Retrieving a Document:

You will get the actual document in ‘_source’ field

In [6]:
res = es.get(index='corpltd', id=3)
print(res)

{'_index': 'corpltd', '_type': '_doc', '_id': '3', '_version': 1, '_seq_no': 13, '_primary_term': 1, 'found': True, '_source': {'first_name': 'rajeev', 'last_name': 'rawat', 'age': 18, 'about': 'I like to build cabinets', 'interests': ['forestry']}}


In [7]:
res['_source']

{'first_name': 'rajeev',
 'last_name': 'rawat',
 'age': 18,
 'about': 'I like to build cabinets',
 'interests': ['forestry']}

# Deleting a document:

In [8]:
res = es.delete(index='corpltd', id=3)
print(res)

{'_index': 'corpltd', '_type': '_doc', '_id': '3', '_version': 2, 'result': 'deleted', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 14, '_primary_term': 1}


In [9]:
res = es.search(index='corpltd', body={'query':{'match_all':{}}})
print('Got {} hits:'.format(res['hits']['total']['value']))

Got 5 hits:


# Search 

In [10]:
res = es.search(index='corpltd', body={'query':{'match_all':{}}})
print(res['hits']['hits'])

[{'_index': 'corpltd', '_type': 'resources', '_id': '4', '_score': 1.0, '_source': {'first_name': 'asd', 'last_name': 'pafdfd', 'age': 27, 'about': 'Love to play football', 'interests': ['sports', 'music']}}, {'_index': 'corpltd', '_type': 'resources', '_id': '5', '_score': 1.0, '_source': {'first_name': 'jay', 'last_name': 'singh', 'age': 27, 'about': 'Love to play cricket', 'interests': ['sports', 'music']}}, {'_index': 'corpltd', '_type': 'resources', '_id': '6', '_score': 1.0, '_source': {'first_name': 'jay', 'last_name': 'singh', 'age': 27, 'about': 'Love to watch cricket', 'interests': ['sports', 'music']}}, {'_index': 'corpltd', '_type': 'resources', '_id': '1', '_score': 1.0, '_source': {'first_name': 'akash', 'last_name': 'rawat', 'age': 30, 'about': 'Love to play tekken', 'interests': ['movies', 'music']}}, {'_index': 'corpltd', '_type': 'resources', '_id': '2', '_score': 1.0, '_source': {'first_name': 'ankit', 'last_name': 'rawat', 'age': 28, 'about': 'I like to collect rock

## match operator:

In [11]:
res = es.search(index='corpltd', body={'query':{'match':{'first_name':'ankit'}}})
print(res['hits']['hits'])

[{'_index': 'corpltd', '_type': 'resources', '_id': '2', '_score': 1.2809337, '_source': {'first_name': 'ankit', 'last_name': 'rawat', 'age': 28, 'about': 'I like to collect rock albums', 'interests': ['music']}}]


# Full text search

In [12]:
e5={
    "first_name":"jay",
    "last_name":"singh",
    "age": 27,
    "about": "Love to play cricket",
    "interests": ['sports','music'],
}

res=es.index(index='corpltd',id=5,body=e5)

In [13]:
res = es.search(index='corpltd', body={'query':{'match':{'about':'cricket'}}})

In [14]:
for hit in res['hits']['hits']:
    print(hit['_source']['about'])
    print(hit['_score'])
    print("*"*20)

Love to play cricket
1.3558909
********************
Love to watch cricket
1.3558909
********************


# Phrase Search
Finding individual words in a field is all well and good, but sometimes you want to match exact sequence of words of phrases.

In [15]:
e6={
    "first_name":"jay",
    "last_name":"singh",
    "age": 27,
    "about": "Love to watch cricket",
    "interests": ['sports','music'],
}

res=es.index(index='corpltd',id=6,body=e6)

In [16]:
res = es.search(index='corpltd', body={'query':{'match_phrase':{'about':'play cricket'}}})

In [17]:
for hit in res['hits']['hits']:
    print(hit['_source']['about'])
    print(hit['_score'])
    print("*"*20)

Love to play cricket
2.0895994
********************


# Aggregations
Elasticsearch has functionality called aggregations, which allowed you to generate sophisticated analytics over your data. It is similar to Group By in SQL, but much more powerful.

In [18]:
res = es.search(index='corpltd', body={'aggs': {'all_interests': {'terms': {'field':'interests.keyword'}}}})
res['aggregations']['all_interests']['buckets']

[{'key': 'music', 'doc_count': 5},
 {'key': 'sports', 'doc_count': 3},
 {'key': 'movies', 'doc_count': 1}]

In [19]:
print(json.dumps(es.indices.get_mapping(index='corpltd'), sort_keys=True, indent=2))

{
  "corpltd": {
    "mappings": {
      "properties": {
        "about": {
          "fields": {
            "keyword": {
              "ignore_above": 256,
              "type": "keyword"
            }
          },
          "type": "text"
        },
        "age": {
          "type": "long"
        },
        "first_name": {
          "fields": {
            "keyword": {
              "ignore_above": 256,
              "type": "keyword"
            }
          },
          "type": "text"
        },
        "interests": {
          "fields": {
            "keyword": {
              "ignore_above": 256,
              "type": "keyword"
            }
          },
          "type": "text"
        },
        "last_name": {
          "fields": {
            "keyword": {
              "ignore_above": 256,
              "type": "keyword"
            }
          },
          "type": "text"
        }
      }
    }
  }
}


# Loan Prediction

In [26]:
import eland as ed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [30]:
train_df = pd.read_csv('loan_train.csv').dropna()
test_df = pd.read_csv('loan_test.csv').dropna()
train_df.shape, test_df.shape

((480, 13), (289, 12))

In [31]:
train_df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y


In [32]:
test_df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban
5,LP001054,Male,Yes,0,Not Graduate,Yes,2165,3422,152.0,360.0,1.0,Urban


In [33]:
def json(x):
    import json
    print(json.dumps(x, indent=2, sort_keys=True))

In [29]:
es = Elasticsearch()

json(es.info())

{
  "cluster_name": "elasticsearch",
  "cluster_uuid": "KDoR-ROET_-UqRewWSB6WA",
  "name": "BLR-DLZXKC2",
  "tagline": "You Know, for Search",
  "version": {
    "build_date": "2020-06-14T19:35:50.234439Z",
    "build_flavor": "default",
    "build_hash": "757314695644ea9a1dc2fecd26d1a43856725e65",
    "build_snapshot": false,
    "build_type": "zip",
    "lucene_version": "8.5.1",
    "minimum_index_compatibility_version": "6.0.0-beta1",
    "minimum_wire_compatibility_version": "6.8.0",
    "number": "7.8.0"
  }
}


In [38]:
train_df.columns = [i.lower() for i in train_df.columns]
test_df.columns = [i.lower() for i in test_df.columns]

train_df.drop('loan_id', axis=1, inplace=True)
test_df.drop('loan_id', axis=1, inplace=True)

train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 1 to 613
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             480 non-null    object 
 1   married            480 non-null    object 
 2   dependents         480 non-null    object 
 3   education          480 non-null    object 
 4   self_employed      480 non-null    object 
 5   applicantincome    480 non-null    int64  
 6   coapplicantincome  480 non-null    float64
 7   loanamount         480 non-null    float64
 8   loan_amount_term   480 non-null    float64
 9   credit_history     480 non-null    float64
 10  property_area      480 non-null    object 
 11  loan_status        480 non-null    object 
dtypes: float64(4), int64(1), object(7)
memory usage: 48.8+ KB


In [39]:
df = ed.pandas_to_eland(pd_df=train_df,
                        es_dest_index='loan_prediction_train',
                        es_client=es,
                       es_if_exists='replace',
                       es_refresh=True)

In [40]:
df.info()

<class 'eland.dataframe.DataFrame'>
Index: 480 entries, 1 to 613
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   applicantincome    480 non-null    int64  
 1   coapplicantincome  480 non-null    float64
 2   credit_history     480 non-null    float64
 3   dependents         480 non-null    object 
 4   education          480 non-null    object 
 5   gender             480 non-null    object 
 6   loan_amount_term   480 non-null    float64
 7   loan_status        480 non-null    object 
 8   loanamount         480 non-null    float64
 9   married            480 non-null    object 
 10  property_area      480 non-null    object 
 11  self_employed      480 non-null    object 
dtypes: float64(4), int64(1), object(7)
memory usage: 80.0 bytes
